# CLASIFICACION IMAGENES

## CAPITULO I. Cargue y transformación de las imagenes

In [1]:
import torchvision.transforms as transforms
import torch.nn as nn

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split


In [3]:
pipeline = transforms.Compose([
    transforms.Grayscale(num_output_channels= 1),   #las imagenes se pasan a escala de grises a 1 solo canal es decir blanco y negro
    transforms.Resize((400, 200)),                  #se usa un tamaño un poco mas grande de lo habitual para tratar de facilitar la deteccion de las palabras
    transforms.RandomHorizontalFlip(),              #se realiza un traslado horizontal
    transforms.RandomRotation(45),                   #rotacion de 45 grados
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))             #normaliza la serie de datos con media 0 y varianza 1
])

baseImagenes = ImageFolder(root="./Imagenes", transform=transforms)

In [ ]:
tamanoEntrenamiento = int(0.8 * len(baseImagenes))
tamanoTesteo = len(baseImagenes) - tamanoEntrenamiento

baseEntrenamiento, baseTesteo = random_split(baseImagenes, [tamanoEntrenamiento, tamanoTesteo])

cargueEntrenammiento = DataLoader(baseEntrenamiento, batch_size=30, shuffle=True)
cargueTesteo = DataLoader(baseTesteo, batch_size=30, shuffle=False)

In [ ]:
class redNeuronal(nn.Module):
    def __init__(self, clases):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), #Entran imagenes a 1 canal se aplican 32 filtros de tamaño 3x3 
            nn.ELU(),                                   #funcion de activacion exponencial
            nn.MaxPool2d(kernel_size=2),                #reduce la dimensionalidad en 2 para mayor eficiencia
            nn.Conv2d(32, 128, kernel_size=3, padding=1), 
            nn.ELU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Flatten(),                               #salida de la red 
        )
        self.classifier = nn.Linear(128 * 32 * 32, clases)
    
    def forward(self, x):  
        x = self.feature_extractor(x)                   #pasa el input por las capas
        x = self.classifier(x)                          #realiza la clasificacion
        return x